In [ ]:
import os
import re
import sys
import glob
import json
import shutil
import pickle
import hashlib
import spectra
import skimage
import datetime
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import ndimage
from skimage import feature
from skimage import morphology

import matplotlib as mpl
import matplotlib.colors
from matplotlib import pyplot as plt

sys.path.append('..')
from opencell.imaging import image, utils, viz
from opencell.imaging import nucleus_segmentation as seg

sys.path.append('/Users/keith.cheveralls/projects/dragonfly-automation/')
from dragonfly_automation import fov_models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
im_dir = '../tests/data/projections/'
paths = glob.glob('%s*.tif' % im_dir)
paths

# 00061 is the hardest
# 0007 is clumpy
# 0076 is overconfluent

In [ ]:
filepath = paths[0]
im = tifffile.imread(filepath)
viz.imshow(utils.autogain(im), figsize=6)

### Canonical background mask

In [ ]:
mask_bg = seg.generate_background_mask(im, sigma=5, rel_thresh=1.0)
# viz.imshow_mask(im, mk=mask_bg, figsize=6, color='red')

### Failed attempts to improve background mask



In [ ]:
# local histogram equalization
# Noisy but seems to capture edges, in a sense, but seems worse and noisier than an LoG filter.
im_eq = skimage.filters.rank.equalize(im, skimage.morphology.disk(15))
im_eqf = skimage.filters.gaussian(im_eq, sigma=1)
im_eqf_mk = im_eqf * mask_bg
viz.imshow(im_eqf_mk[:400, 555:], figsize=12, colorbar=False)

In [ ]:
# local rank filters (grayscale opening)
# this is slow and not promising
im_opening = skimage.filters.rank.minimum(
    skimage.filters.rank.maximum(im[::1, ::1], skimage.morphology.disk(3)), 
    skimage.morphology.disk(3))

In [ ]:
mask_op = generate_background_mask(im_opening, sigma=5, rel_thresh=1.1)

### Improving background mask with LoG

A length scale of `sigma=4` seems to be best.

In [ ]:
mask_lg_f, mask_lg, im_lg = seg.generate_lg_mask(
    im, mask_bg, 
    sigma=4, 
    radius=5, 
    max_area=1e4, 
    percentile=7, 
    min_area=900, 
    debug=True)

viz.imshow(mask_lg_f)

In [ ]:
im_lg = seg.filter_lg(im, 4)
viz.show_mask_props(im_lg < np.percentile(im_lg, 5), 'area')

In [ ]:
viz.imshow_mask(mask_lg_ref, mk=(mask_lg==0), color='red')

In [ ]:
viz.show_mask_props(mask_lg, prop_name='solidity', fmt='%0.3f')

### Create a watershed mask from the background mask

In [ ]:
cmap = sns.color_palette('bright', 10)
cmap_b = [spectra.rgb(*color).desaturate(30).brighten(10).clamped_rgb for color in cmap]

In [ ]:
def viz_seg(mask, mask_bg):
    
    pos = seg.find_nucleus_positions(mask, min_distance=15)
    mask_ws = seg.generate_watershed_mask(mask, min_distance=15, mask_bg=mask_bg)
    
    order = [np.mod(ind, len(cmap)) for ind in range(pos.shape[0])]
    colors = mpl.colors.ListedColormap([(0, 0, 0)] + list(np.array(cmap)[order]))
    im_rgb = skimage.color.label2rgb(
        mask_ws, image=utils.autogain(im, percentile=.3), colors=colors.colors)
    
    viz.imshow(im_rgb, figsize=8)
    plt.scatter(pos[:, 1], pos[:, 0], color='red')
    return mask_ws

In [ ]:
mask_ws = viz_seg(mask_lg_ref, mask_bg=None)

### Show the edges of the watershed mask

In [ ]:
im_ag = utils.autogain(im, percentile=.1).astype(float)[:, :, None]

all_edges = skimage.morphology.dilation(mask_ws > 0) ^ (mask_ws > 0)
all_edges = 200*skimage.morphology.dilation(all_edges, skimage.morphology.disk(1))[:, :, None]

im_rgb = np.concatenate((im_ag + all_edges, im_ag, im_ag), axis=2)
im_rgb[im_rgb > 255] = 255
im_rgb = im_rgb.astype('uint8')
viz.imshow(im_rgb, figsize=8)
tifffile.imsave('%s.seg.tif' % filepath, im_rgb)
tifffile.imsave('%s.ag.tif' % filepath, im_ag.astype('uint8'))

In [ ]:
viz.show_mask_props(mask_labeled, prop_name='solidity', fmt='%0.3f')

In [ ]:
viz.show_mask_props(mask_labeled, prop_name='label', fmt='%d')

### Finding internal edges and counting neighbors

In [ ]:
# example of intersecting edges
viz.show_mask_props(mask_labeled[700:900, 700:900], prop_name='label', fmt='%d')

In [ ]:
# all of the internal watershed edges
edges = (mask_labeled > 0).astype(int) - mask_bg
edges = utils.remove_edge_regions(edges) < 0
edges_l = skimage.morphology.label(edges, connectivity=2)

In [ ]:
def crop_around_region(im, props, pad=0):
    min_row, min_col, max_row, max_col = props.bbox
    min_row = max(0, min_row - pad)
    max_row = min(im.shape[0], max_row + pad)
    min_col = max(0, min_col - pad)
    max_col = min(im.shape[1], max_col + pad)
    return im[min_row:max_row, min_col:max_col]

In [ ]:
# note that counting the number of neighbors here doesn't quite work,
# because it can't identify two distinct edges that intersect
# (and the edges are all 8-connected)
# solution: we need to identify intersection pixels and then remove those before labeling the edges

In [ ]:
label = 50
props = skimage.measure.regionprops(mask_labeled)
prop = [p for p in props if p.label==label][0]

mk = crop_around_region(mask_labeled==label, prop, pad=5)
ed = crop_around_region(edges, prop, pad=5)
ed_l = skimage.measure.label(ed, connectivity=2)

mk_edges = skimage.morphology.dilation(mk > 0) ^ (mk > 0)
viz.imshow((mk > 0).astype(int) + 2*ed + 2*mk_edges, figsize=6)

print('Number of neighbors: %d (relative area: %d%%)' % \
      (len(np.unique(ed_l * mk_edges)) - 1, 100*(ed * mk_edges).sum()/mk_edges.sum()))